## 파일형태에 따른 속도 비교 

In [ ]:
import numpy as np
import pandas as pd

pd.options.display.float_format = '{:,.4f}'.format

In [ ]:
# random으로 100,000개 관측개체 X 50 변수
난수개수 = 100000
변수개수 = 50
values = np.random.random((난수개수,변수개수))
df = pd.DataFrame(values)
df

### Padas DataFrame
- info(): DF에 대한 기본 정보(행과 열의 수, 각 열의 데이터 타입 등) 
- shape: 행과 열의 수
- dtypes: 변수(열)의 데이터 형식
- columns: 변수(열)명
    - 전체 열 이름 변경: df.columns=[ ....]
    - 열 이름 부분 변경: df.rename(columns={'0':'A','1':'B'},inplace=True)
- head(): 앞 부분 5개 return (개수 조정 가능)  $\longrightarrow$ 앞 부분 데이터 선택
- tail(): 뒷 부분 5개 return (개수 조정 가능)  $\longrightarrow$ 뒷 부분 데이터 선택
- loc: index(행)이름 해당 또는 T/F 값에 따른 데이터 선택
- iloc, ix: 번호로 행/열의 데이터 선택
- query(): 열들에 대해 boolean 연산결과로 rows 선택
    - Ex: df.query('A > B')   # df[df.A > df.B]와 동일

#### columns 이름 변경
- 숫자 앞에 'X' 추가

In [ ]:
df.columns = list(map(lambda x:"X"+str(x), range(변수개수)))
df.info()

### 작업 속도 비교
timeit으로 실행 측정한 시간을 비교

In [ ]:
# 결과를 저장할 빈 데이터 프레임 생성
df_result = pd.DataFrame(columns=['read', 'write'])
df_result.head()

In [ ]:
# !pip install openpyxl 
# 시간이 많이 걸리기 때문에 실습에서는 제외
# Excel : 행수 1,048,576개, 열수 16,384개로 제한  
excel = %timeit -o df.to_excel('test_data.xlsx')
df_result.loc['excel', 'write'] = excel.best
excel = %timeit -o df = pd.read_excel('test_data.xlsx')
df_result.loc['excel', 'read'] = excel.best

In [ ]:
# CSV
csv = %timeit -o df.to_csv('test_data.csv')
df_result.loc['csv0', 'write'] = csv.best
csv0 = %timeit -o pd.read_csv('test_data.csv')
df_result.loc['csv0', 'read'] = csv0.best

In [ ]:
# JSON
json = %timeit -o df.to_json('test_data.json')
df_result.loc['json', 'write'] = json.best
json = %timeit -o pd.read_json('test_data.json')
df_result.loc['json', 'read'] = json.best

In [ ]:
# pickle
pickle = %timeit -o df.to_pickle('test_data.pickle')
df_result.loc['pickle', 'write'] = pickle.best
pickle = %timeit -o pd.read_pickle('test_data.pickle')
df_result.loc['pickle', 'read'] = pickle.best

In [ ]:
# feather
# pyarrow 패키지 필요: !pip install pyarrow
feather = %timeit -o df.to_feather('test_data.ftr')
df_result.loc['feather', 'write'] = feather.best
feather = %timeit -o pd.read_feather('test_data.ftr')   #옵션: dtype_backend='pyarrow'
df_result.loc['feather', 'read'] = feather.best

#### Parquet(파케이): 
- 많은 빅데이터 처리  프레임 워크 및 플랫폼 지원
- 다양한 I/O 최적화를 제공하는 오픈소스 칼럼 기반 파일 형식

In [ ]:
parquet = %timeit -o df.to_parquet('test_data.parquet')
df_result.loc['parquet', 'write'] = parquet.best
parquet = %timeit -o pd.read_parquet('test_data.parquet')  #옵션: engine='pyarrow', default = 'auto'
df_result.loc['parquet', 'read'] = parquet.best

In [ ]:
# CSV+engine='pyarrow'
# csv0 = %timeit -o df.to_csv('test_data.csv')
df_result.loc['csv1', 'write'] = csv.best
csv1 = %timeit -o pd.read_csv('test_data.csv', engine='pyarrow')
df_result.loc['csv1', 'read'] = csv1.best

In [ ]:
# CSV+engine='pyarrow'+dtype_backend="pyarrow'
# csv0 = %timeit -o df.to_csv('test_data.csv')
df_result.loc['csv2', 'write'] = csv.best
csv2 = %timeit -o pd.read_csv('test_data.csv', engine='pyarrow',dtype_backend='pyarrow')
df_result.loc['csv2', 'read'] = csv2.best

In [ ]:
df_result

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (10,5)
plt.rcParams["font.size"] = 20

In [ ]:
df_result
df_result.plot(kind='bar')
plt.show()

### Spark 기본 데이터 소스
- Parquet files
- ORC files
- JSON files
- CSV files
- Text files
- Hive tables
- JDBC to other databases
- Avro files
- Binary file: Ex, image file